In [7]:
# Imports
import os
import json
import tiktoken
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, QueryType, QueryCaptionType, QueryAnswerType, VectorFilterMode
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
false = True #Set to true to see more output information
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
model = "ada002"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

In [8]:
rag_content=""
# system message
system_message = f'''
You are an assistant with knowledge of the following topics:
1. IEC61131-3 languages
2. Structured Text
3. Function Block Diagram
4. IEC61131-3 coding standards
5. IEC61131-3 best practices
6. IEC61131-3 coding guidelines
7. IEC61131-3 programming
8. IEC61131-3 programming languages
9. Schneider Electric EcoStruxure Control Expert
10. Schneider Electric EcoStruxure Machine Expert
11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

Your job is to generate small examples of code using exclusiveliy IEC61131-3 Structured Text base on user input.
You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

'''

In [9]:
def get_user_command(lib_name:str, fn_name:str, rag_info:str):
  return f'''  
    Generate a small program in IEC61131-3 that uses Scheneider Electric {lib_name} library to send an email using following parameters:
    1. To: "receiver@se.com"
    2. Subject: "Test email"
    3. Body: "This is a test email"
    4. From: "sender@se.com"

    Authentication required.
    Message should be sent with high priority.
    Use Login to authenticate using "corrado" as user and "p@ssw0rd123" as password.
    Verify that the email has been sent successfully, if not print the error message.

    These are the information about the {fn_name} function from the {lib_name} library you have to use:
    ```
    {rag_info}
    ```
    '''

Following cell uses LLM to summarize RAG content, but looks like is not working since it strips out most of the important info.

In [10]:
# messages=[]
# messages.append({'role': 'system', 'content': "You are an assistant expert in summarize code libraries documentation. Your goal is to summarize the user provided content removing not relevant information"})
# messages.append({'role': 'user', 'content': rag_content})  

# token_count=count_tokens(rag_content);
# print (f'Total input RAG tokens: {token_count}')     

# openai_response = client.chat.completions.create(
#         model=deployment,    
#         messages = messages,
#         temperature=0.3,
#         max_tokens=800,
#         top_p=0.95,
#         frequency_penalty=0,
#         presence_penalty=0,
#         stop=None)

# rag_content= openai_response.choices[0].message.content

# token_count=count_tokens(rag_content);
# print (f'Summarized RAG tokens: {token_count}') 
# messages=[]

# # system message
# system_message = f'''
# You are an assistant with knowledge of the following topics:
# 1. IEC61131-3 languages
# 2. Structured Text
# 3. Function Block Diagram
# 4. IEC61131-3 coding standards
# 5. IEC61131-3 best practices
# 6. IEC61131-3 coding guidelines
# 7. IEC61131-3 programming
# 8. IEC61131-3 programming languages
# 9. Schneider Electric EcoStruxure Control Expert
# 10. Schneider Electric EcoStruxure Machine Expert
# 11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

# Your job is to generate small examples of code using exclusiveliy IEC61131-3 Structured Text base on user input.
# You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

# Use the following pieces of retrieved context to answer the question.
# CONTEXT:
# ```
# {rag_content}
# ```

# '''

# print(system_message)


In [11]:

def generate_embeddings(text):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_from_RAG(query: str, max_results: int = 10, verbose: bool = False):    
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    vector_query = VectorizedQuery(vector= generate_embeddings(query), k_nearest_neighbors=max_results, fields="embedding")
    results = search_client.search(          
        vector_queries=[vector_query],
        select=["id","sourcefile","content"],
        query_type=QueryType.SEMANTIC, 
        top=max_results,
        semantic_configuration_name='default', 
        query_caption=QueryCaptionType.EXTRACTIVE, 
        query_answer=QueryAnswerType.EXTRACTIVE        
    )   
    
    rag_results=[]
    if results :        
        for result in results:
            if verbose:            
                print(f"Id: {result['id']}")
                print(f"Reranker Score: {result['@search.score']}")
                print(f"Content: {result['content']}")
                print(f"Sourcefile: {result['sourcefile']}")                
            rag_results.append(result['content'])                
                
    return rag_results

Theses are the user commands, edit and run them to see the different output considering that output also depends on code implemented into function (to be replaced by RAG)

In [12]:
lib_name="EmailHandling"
fn_name="FB_SendEmail"

query = f"Find all the info related to the {fn_name} function in the {lib_name} library."
rag_results = get_from_RAG(query, max_results=5, verbose=False)
rag_data='n'.join(rag_results)

token_count=count_tokens(rag_data);
print (f'Total rag tokens: {token_count}')  

# uncomment to remove rag_data
#rag_data=""
  
user_command=get_user_command(lib_name, fn_name, rag_data)

messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': user_command})  

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
result= openai_response.choices[0].message

print (result.content)

Total rag tokens: 1093
Here is an example of how you can use the FB_SendEMail function block from the EmailHandling library to send an email:

```iecst
PROGRAM SendEmail
VAR
    EmailClient: SE_EMail.FB_SendEMail;
    EmailCredentials: SE_EMail.ST_Credentials;
    EmailMessage: SE_EMail.ST_Message;
    EmailError: SE_EMail.ST_Error;
    SendEmail: BOOL;
END_VAR
```

In the initialization part of your program, you can set up the email credentials and message:

```iecst
EmailCredentials.sUser := 'corrado';
EmailCredentials.sPassword := 'p@ssw0rd123';

EmailMessage.sFrom := 'sender@se.com';
EmailMessage.sTo := 'receiver@se.com';
EmailMessage.sSubject := 'Test email';
EmailMessage.sBody := 'This is a test email';
EmailMessage.ePriority := SE_EMail.E_Priority#high;

SendEmail := TRUE;
```

Then, in the cyclic part of your program, you can use the FB_SendEMail function block to send the email:

```iecst
EmailClient(
    xEnable := SendEmail,
    stCredentials := EmailCredentials,
    pbyMess